# Parallel Configuration

This Jupyter notebook will walk you through using threads to execute commands that configure your experiment. 


## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Create the Experiment Slice

The following creates a set of 10 nodes with basic compute capabilities on random sites.


In [ ]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice")

    for i in range(10):
        # Add a node
        node = slice.add_node(name=f"Node{i}")

    #Submit the Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the node

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Configure the Nodes in Parallel


In [ ]:
try:
    config_command = "sudo yum install -q -y net-tools"
    
    #Create execute threads
    execute_threads = {}
    for node in slice.get_nodes():
        print(f"Starting config on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
        print(f"stdout: {stdout}")
        #print(f"stderr: {stderr}")

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    for node in slice.get_nodes():
        stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s` && netstat -i')
        print(stdout)
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")